LRU 算法

https://labuladong.github.io/article/fname.html?fname=LRU算法

In [ ]:
class Node: # 双链表的节点类
    def __init__(self, k: int, v: int) -> None:
        self.key = k
        self.val = v
        self.next = None
        self.prev = None

In [ ]:
class DoubleList: # 构建一个双链表，实现几个 LRU 算法必须的 API
    # 头尾虚节点
    def __init__(self) -> None:
        self.head = Node(0, 0)
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head
        self.size = 0

    # 在链表尾部添加节点 x，时间 O(1)
    def addLast(self, x: Node):
        x.prev = self.tail.prev
        x.next = self.tail
        self.tail.prev.next = x
        self.tail.prev = x
        self.size += 1

    # 删除链表中的 x 节点（x 一定存在），时间 O(1)
    def remove(self, x: Node):
        x.prev.next = x.next
        x.next.prev = x.prev
        self.size -= 1

    # 删除链表中第一个节点，并返回该节点，时间 O(1)
    def removeFirst(self) -> Node:
        pass

In [ ]:
# 146. LRU 缓存
# https://leetcode.cn/problems/lru-cache/

class LRUCache:

    def __init__(self, capacity: int):
        pass

    def get(self, key: int) -> int:
        pass

    def put(self, key: int, value: int) -> None:
        pass


# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)